In [1]:
import numpy as np
# from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.pso import PSO
from scipy.interpolate import griddata
from pymoo.termination.default import DefaultSingleObjectiveTermination
import pandas as pd


In [2]:
jpeg_cha = pd.read_csv("../../stable_tests/measurements/jpeg_snr_cha/characteristic.csv")

jpeg_cha = jpeg_cha[jpeg_cha["sparsity"]>0]
jpeg_cha = jpeg_cha[jpeg_cha["datasize_est"]>0]
tensor_size = 128*26*26 *4
jpeg_cha["ratio"] = tensor_size/jpeg_cha["datasize_est"]

In [3]:
pruning = [0.05, 0.1, 0.15, 0.2, 0.25,0.3,0.35]
quality = [60,70,80,90,100]

sample_points=[]
cmp_sample_values = []
snr_sample_values = []

cha_df_group =jpeg_cha.groupby("pruning_thresh")
for p in pruning:
    cha_df = cha_df_group.get_group(p)
    cha_quality_df = cha_df.groupby("quality")
    for q in quality:
        cha_plot_df= cha_quality_df.get_group(q)
        sample_points.append([q,p])
        cmp_sample_values.append(cha_plot_df["ratio"].mean())
        snr_sample_values.append(cha_plot_df["reconstruct_snr"].mean())

sample_points = np.array(sample_points)
cmp_sample_values = np.array(cmp_sample_values)
snr_sample_values = np.array(snr_sample_values)

In [4]:
# Interpolate the constraint function
def interpolated_constraint_cmp(xy):
    return griddata(sample_points, cmp_sample_values, (xy[0], xy[1]/100), method='cubic')

def interpolated_constraint_snr(xy):
    return griddata(sample_points, snr_sample_values, (xy[0], xy[1]/100), method='cubic')


class CustomProblem(ElementwiseProblem):

    def __init__(self,snr,cmp):
        self.snr = snr
        self.cmp = cmp
        super().__init__(n_var=2, n_obj=1, n_ieq_constr=2, xl=[60,5], xu=[100,35])

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = x[1]/35-x[0]/100

        constraint_1 = self.cmp-interpolated_constraint_cmp(x)
        constraint_2 = self.snr-interpolated_constraint_snr(x)

        out["G"] = np.array([
            constraint_1,constraint_2
        ])
        penalty = 1000  # Large penalty factor
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_1
        if constraint_1 > 0:  # If constraint is violated
            out["F"] += penalty * constraint_2

In [8]:
# Initialize the problem
problem = CustomProblem(snr=25,cmp=10)

# Select the algorithm (Genetic Algorithm in this case)
algorithm = PSO(pop_size=5)

termination = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=10,
    n_max_evals=10000
)


# Perform the optimization
result = minimize(problem, algorithm, termination=termination,verbose=True)

# # Get and display the results
# optimal_x, optimal_y = result.X
# optimal_objective_value = -result.F[0]

print("Best solution found: \nX = %s\nF = %s" % (result.X, result.F))

n_gen  |  n_eval  |    f     |    S    |    w    |    c1    |    c2    |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |        5 |        - |       - |  0.9000 |  2.00000 |  2.00000 |  2.7634392029 |  1.553355E+01 |             - |             -
     2 |       10 | -3.6E-01 |       3 |  0.2086 |  2.00000 |  2.02280 |  2.7634392029 |  1.179227E+01 |             - |             -
     3 |       15 | -1.1E-01 |       3 |  0.3312 |  1.98892 |  2.03990 |  0.000000E+00 |  7.9970582053 | -7.466866E-01 | -7.466866E-01
     4 |       20 | -3.0E-01 |       3 |  0.2318 |  1.97529 |  2.05545 |  0.000000E+00 |  2.8989144411 | -7.158903E-01 | -7.871082E-01
     5 |       25 | -2.9E-01 |       3 |  0.2398 |  1.96115 |  2.07009 |  0.000000E+00 |  0.9002684302 | -7.490636E-01 | -7.956448E-01
     6 |       30 | -1.0E-01 |       3 |  0.3383 |  1.94681 |  2.07699 |  0.000000E+00 |  0.000000E+00 | -7.624098E-01 | -7.956448E-01
     7 |       35 | -1.8E-01 |       3 |  0.2955 |  1.9

In [6]:
interpolated_constraint_cmp([82,20])

array(107.50481253)

In [7]:
interpolated_constraint_snr([82,20])

array(5.18968897)